In [180]:
import tensorflow as tf
from tensorflow import keras
import os
import sys
import matplotlib.pyplot as plt

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [181]:
glioma_path = '/Users/alex/Desktop/NeuroRex/data/mri_tumor_classifier/glioma'
meningioma_path = '/Users/alex/Desktop/NeuroRex/data/mri_tumor_classifier/meningioma'
pituitary_path = '/Users/alex/Desktop/NeuroRex/data/mri_tumor_classifier/pituitary'
notumor_path = '/Users/alex/Desktop/NeuroRex/data/mri_tumor_classifier/notumor'

In [182]:
def decode_image(file_path, image_data):
    ext = tf.strings.lower(tf.strings.split(file_path, '.')[-1])
    
    def decode_jpg():
        return tf.image.decode_jpeg(image_data, channels=3)
    
    def decode_png():
        return tf.image.decode_png(image_data, channels=3)
    
    return tf.case([
        (tf.equal(ext, 'jpeg'), decode_jpg),
        (tf.equal(ext, 'jpg'), decode_jpg),
        (tf.equal(ext, 'png'), decode_png),
    ], exclusive=True, default=decode_jpg)

In [183]:
IMG_SIZE = (224, 224)

def process_path(file_path, label):
    image_data = tf.io.read_file(file_path)
    image = decode_image(file_path, image_data)
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)/255.0
    
    return image, label

In [184]:
CLASS_LABELS = {
    'glioma':0,
    'meningioma':1,
    'pituitary':2,
    'notumor':3
}
glioma_files = [os.path.join(glioma_path, f) for f in os.listdir(glioma_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
meningioma_files = [os.path.join(meningioma_path, f) for f in os.listdir(meningioma_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
pituitary_files = [os.path.join(pituitary_path, f) for f in os.listdir(pituitary_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
notumor_files = [os.path.join(notumor_path, f) for f in os.listdir(notumor_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]

In [185]:
all_files = glioma_files+meningioma_files+pituitary_files+notumor_files
all_labels = len(glioma_files)*[0]+len(meningioma_files)*[1]+len(pituitary_files)*[2]+len(notumor_files)*[3]

In [186]:
from sklearn.model_selection import train_test_split

train_data, temp_data, train_labels, temp_labels = train_test_split(all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels)
val_data, test_data, val_labels, test_labels = train_test_split(temp_data, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels)

In [187]:
def make_dataset(data, labels, shuffle=True, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [188]:
train_ds = make_dataset(train_data, train_labels)
val_ds = make_dataset(val_data, val_labels, shuffle=False)
test_ds = make_dataset(test_data, test_labels, shuffle=False)

In [189]:
from models.mri_tumor_classifier.cnn_model import mri_tumor_classifier

model = mri_tumor_classifier()


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
 3 (GlobalAveragePooling2D)                                      
                                                                 
 dropout_13 (Dropout)        (None, 1280)              0         
                                                                 
 dense_26 (Dense)            (None, 128)               163968    
                                                                 
 dense_27 (Dense)            (None, 4)                 516

In [190]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_names = [0, 1, 2, 3]
unique_classes = np.unique(train_labels)
class_weight = compute_class_weight(class_weight='balanced', classes=unique_classes, y = train_labels)

In [191]:
class_weight_dict = dict(enumerate(class_weight))
print(class_weight_dict)

{0: 1.0828835774865073, 1: 1.0672492401215805, 2: 0.999644128113879, 3: 0.8778125}


In [192]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(1e-3),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [193]:
history_1 = model.fit(
    train_ds, validation_data=val_ds, epochs = 5, class_weight=class_weight_dict
)

Epoch 1/5
176/176 [==============================] - 37s 187ms/step - loss: 0.5123 - accuracy: 0.8120 - val_loss: 0.2645 - val_accuracy: 0.9003
Epoch 2/5
176/176 [==============================] - 35s 196ms/step - loss: 0.3059 - accuracy: 0.8841 - val_loss: 0.2127 - val_accuracy: 0.9259
Epoch 3/5
176/176 [==============================] - 43s 237ms/step - loss: 0.2896 - accuracy: 0.8946 - val_loss: 0.2160 - val_accuracy: 0.9117
Epoch 4/5
176/176 [==============================] - 52s 285ms/step - loss: 0.2516 - accuracy: 0.9082 - val_loss: 0.2268 - val_accuracy: 0.9145
Epoch 5/5
176/176 [==============================] - 55s 301ms/step - loss: 0.2321 - accuracy: 0.9156 - val_loss: 0.1550 - val_accuracy: 0.9416


In [194]:
model.evaluate(test_ds)

22/22 [==============================] - 7s 307ms/step - loss: 0.2055 - accuracy: 0.9203


[0.20549143850803375, 0.9203413724899292]